In [ ]:
# default_exp exposure
%load_ext autoreload
%autoreload 2

# Exposure
> calculate exposure for a point source from a GTI and FT2 files

Main interface is `get_binned_exposure`, which returns arrays with the (default) bins and exposure, needed for making cells.

Also manage default bins, see `get_default_bins`. 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from astropy.io import fits
import numpy as np
import pandas as pd
from scipy.integrate import simps

from wtlike.config import *
from wtlike.load_gti import get_gti
from wtlike.effective_area import EffectiveArea

In [ ]:
#export
def get_default_bins(config, exposure):
    """set up default bins from exposure; and config.mjd_range if set.
    
    adjust stop to come out even,    round to whole day
    """

    start = np.round(exposure.start.values[0])
    stop =  np.round(exposure.stop.values[-1])
    if config.mjd_range is None:
        config.mjd_range = (start,stop)

    step = config.time_interval
    nbins = int(round((stop-start)/step))
    tb =time_bins = np.linspace(start, stop, nbins+1)
    if config.verbose>0:
        print(f'Time bins: {nbins} intervals of {step} days, '\
              f'in range ({time_bins[0]:.1f}, {time_bins[-1]:.1f})')
    return time_bins

In [ ]:
#export
def _process_ft2(config, source, ft2_file_path, gti, effective_area):
    """Process a set of FT2 files, with S/C history data
    Parameters:
        - config -- verbose, cos_theta_max, z_max
        - source -- l,b for position
        - ft2_files -- list of spacecraft files
        - gti -- GTI object with allowed intervals
        - effective_area -- function of energy and angle with respect to zenity
    Generate a dataset with fields:
        - start, stop : start  and stop times in MJD
        - exposure    : calculated exposure using effective area

     """
    # combine the files into a DataFrame with following fields besides START and STOP (lower case for column)
    fields    = ['LIVETIME','RA_SCZ','DEC_SCZ', 'RA_ZENITH','DEC_ZENITH']
    ft2_files = list(ft2_file_path.glob('*.fits'))
    if config.verbose>1:
        print(f'Processing {len(ft2_files)} S/C history (FT2) files')
        print(f'  applying cuts cos(theta) < {config.cos_theta_max},  z < {config.z_max}')
    sc_data=[]
    for filename in ft2_files:
        with fits.open(filename) as hdu:
            scdata = hdu['SC_DATA'].data
            # get times to check against MJD limits and GTI
            start, stop = [MJD(np.array(scdata.START, float)),
                           MJD(np.array(scdata.STOP, float))]
            if config.mjd_range is not None:
                a,b=  config.mjd_range
                if stop[-1]<a:
                    print(f'\tfile {filename}: skip, before selected range' )
                    continue
                elif start[0]>b:
                    print(f'\tfile {filename}: quit, beyond range')
                    break
            # apply GTI to bin center (avoid edge effects?)
            in_gti = gti(0.5*(start+stop))
            if config.verbose>2:
                print(f'\tfile {filename}: {len(start)} entries, {sum(in_gti)} in GTI')
            t = [('start', start[in_gti]), ('stop',stop[in_gti])]+\
                [(field.lower(), np.array(scdata[field][in_gti],np.float32)) for field in fields ]
            sc_data.append( pd.DataFrame(dict(t) ) )
    df = pd.concat(sc_data, ignore_index=True)

    # calculate cosines with respect to sky direction
    sc = source
    ra_r,dec_r = np.radians(sc.ra), np.radians(sc.dec)
    sdec, cdec = np.sin(dec_r), np.cos(dec_r)

    def cosines( ra2, dec2):
        ra2_r =  np.radians(ra2.values)
        dec2_r = np.radians(dec2.values)
        return np.cos(dec2_r)*cdec*np.cos(ra_r-ra2_r) + np.sin(dec2_r)*sdec

    pcosines = cosines(df.ra_scz,    df.dec_scz)
    zcosines = cosines(df.ra_zenith, df.dec_zenith)

    # mask out entries too close to zenith, or too far away from ROI center
    mask =   (pcosines >= config.cos_theta_max) & (zcosines>=np.cos(np.radians(config.z_max)))
    if config.verbose>1:
        print(f'\tFound {len(mask):,} S/C entries:  {sum(mask):,} remain after zenith and theta cuts')
    dfm = df.loc[mask,:]
    livetime = dfm.livetime.values
    config.dfm = dfm ##############debug
    # apply MJD range if present. note times in MJD
    start, stop = dfm.start,dfm.stop
    lims = slice(None)
    if config.mjd_range is not None:
#         a, b = config._get_limits(start)
        a, b = np.searchsorted(start, config.mjd_range)
        if a>0 or b<len(start):
            if config.verbose>1:
                print(f'\tcut from {len(start):,} to {a} - {b}, or {b-a:,} entries after MJD range selection')
            dfm = dfm.iloc[a:b]
            lims = slice(a,b)


    expose = _exposure(config, effective_area, livetime[lims], pcosines[mask][lims])
    return pd.DataFrame(dict(start=start[lims],stop=stop[lims], exposure=expose))

In [ ]:
#export
def _exposure(config, effective_area, livetime, pcosine):
    """return exposure calculated for each pair in livetime and cosines arrays

    uses effective area
    """
    assert len(livetime)==len(pcosine), 'expect equal-length arrays'

    # get a set of energies and associated weights from a trial spectrum

    emin,emax = config.energy_range
    loge1=np.log10(emin); loge2=np.log10(emax)

    edom=np.logspace(loge1, loge2, int((loge2-loge1)*config.bins_per_decade+1))
    if config.verbose>1:
        print(f'Calculate exposure using the energy domain'\
              f' {emin}-{emax} {config.bins_per_decade} bins/decade' )
    base_spectrum = eval(config.base_spectrum) #lambda E: (E/1000)**-2.1
    assert base_spectrum(1000)==1.
    wts = base_spectrum(edom)

    # effectivee area function from
    ea = effective_area

    # a table of the weighted for each pair in livetime and pcosine arrays
    rvals = np.empty([len(wts),len(pcosine)])
    for i,(en,wt) in enumerate(zip(edom,wts)):
        faeff,baeff = ea([en],pcosine)
        rvals[i] = (faeff+baeff)*wt

    aeff = simps(rvals,edom,axis=0)/simps(wts,edom)
    return (aeff*livetime)

In [ ]:
#export
def get_exposure(config,  source, gti_key='gti'):
    """Return the exposure for the source

    """

    


        
    def getit():
        files = config.files
   
        gti = get_gti(config, gti_key)
        aeff = EffectiveArea(file_path = files.aeff)
        exposure =  _process_ft2(config, source, files.ft2, gti, aeff)
        if config.verbose>1:
            print(f'{len(exposure)} entries, MJD {exposure.iloc[0].start:.0f}'
                  f' - {exposure.iloc[-1].stop:.0f}')
        return exposure
    
    key = f'exposure_{source.name}'
    description = f'exposure for {source.name}' if config.verbose>1 else ''
    return config.cache(key, getit, description=description)


Get, and display the exposure:

The result is a tuple of time intervals and the exposure

In [ ]:

from wtlike.config import Config,  PointSource, Cache
config = Config()
source = PointSource('Geminga')

if config.valid:
    exposure = get_exposure(config, source)  
    print(f'Exposure for {source.name}:\n {exposure}')


exposure for Geminga: Restoring from cache with key "exposure_Geminga"
Exposure for Geminga:
                 start          stop       exposure
0        54682.656038  54682.656375  112998.052089
1        54682.656375  54682.656722  116901.160578
2        54682.656722  54682.657069  116857.143435
3        54682.657069  54682.657416  116791.434285
4        54682.657416  54682.657764  116876.550059
...               ...           ...            ...
9609753  58698.052602  58698.052949   52427.727986
9609754  58698.052949  58698.053296   49343.687807
9609755  58698.053296  58698.053643   44567.360241
9609756  58698.053643  58698.053991   38779.120217
9609757  58698.053991  58698.054338   31786.491045

[2695715 rows x 3 columns]


In [ ]:
#export
def get_contiguous_exposure(exposure, max_interval=10, verbose=0 ):
    """Combine exposure intervals from an exposure dataframe
    return array of (start, stop) pairs
    """

    t0s = exposure.start.values
    t1s = exposure.stop.values
    break_mask = (t0s[1:]-t1s[:-1])>max_interval/day 
    break_starts = t1s[:-1][break_mask]
    break_stops = t0s[1:][break_mask]
    # now assumble the complement
    good_starts = np.empty(len(break_starts)+1)
    good_stops = np.empty_like(good_starts)
    good_starts[0] = t0s[0]
    good_starts[1:] = break_stops
    good_stops[-1] = t1s[-1]
    good_stops[:-1] = break_starts
    if verbose>1:
        print(f'Generate list of contiguous exposures:\n'\
              f'  WIth max interval {max_interval} s, combine {len(t0s):,} exposure entries to {len(good_stops):,} ')
    return np.array([good_starts, good_stops]).T

In [ ]:
#export
def get_binned_exposure(config, source, time_bins=None, key='', gti_key='gti'):
    """
    
    Calculate integrated exposures per bin
    
    - `source` -- a PointSource object
    
    - `time_bins` -- a list of bins, or None<br> If None, generate the default
    
    - `key` -- use cache unless set to None. 
    
    Return two arrays: a list of integrated exposures per bin, normalized to the average per bin,
    and the time_bin array.
    """

    def doit(time_bins):
        # get exposure calculation
        exposure = get_exposure(config, source, gti_key)
        exp   = exposure.exposure.values
        estart= exposure.start.values
        estop = exposure.stop.values

        #use cumulative exposure to integrate over larger periods
        cumexp = np.concatenate(([0],np.cumsum(exp)) )

        # default bins depends on exposure
        if time_bins is None:
            time_bins = get_default_bins(config, exposure)

        # get index into tstop array of the bin edges
        edge_index = np.searchsorted(estop, time_bins)
        # return the exposure integrated over the intervals
        cum = cumexp[edge_index]

        return np.diff(cum)/(cum[-1]-cum[0]) * (len(time_bins)-1) , time_bins
    
    key = f'binexp_{source.name}' if key=='' else key
    description= f'binned exposure for source {source.name}'
    
    return config.cache(key, doit, time_bins, description=description)
    

In [ ]:

if config.valid:
    be, bins = get_binned_exposure(config, source,  key=None)
    print(f'Binned exposure:\n{be}, \ntime bins: {bins}')
    assert len(be)==len(bins)-1


exposure for Geminga: Restoring from cache with key "exposure_Geminga"
Time bins: 4015 intervals of 1 days, in range (54683.0, 58698.0)
Binned exposure:
[1.26455999 1.21603611 1.18307691 ... 0.91251873 0.57874049 0.00704994], 
time bins: [54683. 54684. 54685. ... 58696. 58697. 58698.]


In [ ]:
# #hide
if config.valid:
    expsub = exposure#.iloc[:1000]
    ec = get_contiguous_exposure(expsub, verbose=2)
 
    a, b  = ec[:,0], ec[:,1]
    gap = b[1:]-a[:-1]

    delta = (b-a)
    print(f'Check exposures in hours, gap:  {gap.min()*24:.2f}-{gap.max()*24:.2f}'\
          f'  delta: {delta.min()*24:.2f}- {delta.max()*24:.2f}')
    #plt.hist(delta*24*60, np.linspace(0,40,21));
    

Generate list of contiguous exposures:
  WIth max interval 10 s, combine 2,695,715 exposure entries to 61,561 
Check exposures in hours, gap:  0.02-751.07  delta: 0.00- 0.93


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 06_poisson.ipynb.
Converted 07_cells.ipynb.
Converted 08_loglike.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10_simulation.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Sun Dec 27 04:40:33 PST 2020
